# Reproducing in-text result 2

This result starts with the doctor utilisation from Figure 2A which is over 100% when patient load is 170 (IAT 3) and appointment times are 5 minutes (mean 5, SD 1, boundary 2).

The variant introduced is to give all administrative work to the staff nurse.

## Set up

In [1]:
# To run model
import PHC
from reproduction_helpers import process_results

# To import results and produce figures
import xlrd
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

In [2]:
# Paths to save image files to
output_folder = 'outputs'
save_path = os.path.join(output_folder, 'intext2.csv')

## Run model

In [3]:
# Parameters used in both models
base_model = {
    'OPD_iat': 3,
    'rep_file': 'arr170',
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2
}

# Model variants
variants = [
    {
        'admin_to_staff_nurse': False,
        'rep_file': 'in2_admin_doctor.xls'
    },
    {
        'admin_to_staff_nurse': True,
        'rep_file': 'in2_admin_nurse.xls'
    },
]

In [4]:
# Combine dictionaries
dict_list = []
for var in variants:
    dict_list.append({**base_model, **var})

# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

# View dictionaries
dict_list

[{'s_OPD_iat': 3,
  's_rep_file': 'in2_admin_doctor.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_admin_to_staff_nurse': False},
 {'s_OPD_iat': 3,
  's_rep_file': 'in2_admin_nurse.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_admin_to_staff_nurse': True}]

In [5]:
'''# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)'''

'# Wrapper function to allow input of dictionary with pool\ndef wrapper(d):\n    return PHC.main(**d)\n\n# Create a process pool that uses all CPUs\nwith Pool() as pool:\n    # Run PHC.main() using each of inputs from config\n    pool.map(wrapper, dict_list)'

## Process results

In [6]:
# Process results
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)

# Get utilisation
util = round(pd.DataFrame(data.loc['doc occ']).T, 3)

# Find change
util['Change'] = util['in2_admin_doctor'] - util['in2_admin_nurse']

# Rename columns for clarity for readers
util = util.rename({
    'in2_admin_doctor': 'Original doctor utilisation',
    'in2_admin_nurse': (
        'Doctor utilisation if admin is assigned to the staff nurse')
}, axis=1)

# Save and view results
util.to_csv(save_path, index=False)
util

,Original doctor utilisation,Doctor utilisation if admin is assigned to the staff nurse,Change
doc occ,1.142,1.023,0.119
